# Probabilities for three level nests differ

Asim scales nested utilties differently, for two level and RU2 it's identical to what we do (which agrees with the
larch paper and Zenith), but for three levels there are small differences of probabilities and logsums. I
quadruple-checked the scaling and leaf and note utility calculation, let's use larch to see if we get the same as
Asim or if the results agree with my implementation. The idea is to use estimation mode to load a small data set, and
 then use my Asim setup in validate_frozen_impl.ipynb to calculate Asim's and my probabilities by sampling, and check
  what larch has to say by using m.probabilities() (no need to estimate, use parameters as given by asim model).


In [104]:
import os
import sys

import numpy as np
import pandas as pd

# I installed larch deps (which I collected in a requirements file, xmle and sharrow are not on pypi)
# python -m pip install -r ../larch/requirements.txt

larch_path = "/mnt/c/Users/jan.zill/code/larch"
if larch_path not in sys.path:
    sys.path.append(larch_path)
import larch

In [105]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_estimation_sf")
os.chdir(example_dir)

In [106]:
# create estimation test example
#!activitysim create -e example_estimation_sf -d test_estimation_sf

In [13]:
# run estimation mode TODO: smaller data set?
!activitysim run -c configs_estimation/configs -c configs -o output -d data_sf

Configured logging using basicConfig
INFO:activitysim:Configured logging using basicConfig
INFO - Read logging configuration from: configs_estimation/configs/logging.yaml
INFO - SETTING configs_dir: ['configs_estimation/configs', 'configs']
INFO - SETTING settings_file_name: settings.yaml
INFO - SETTING data_dir: ['data_sf']
INFO - SETTING output_dir: output
INFO - SETTING households_sample_size: 0
INFO - SETTING chunk_size: 0
INFO - SETTING chunk_method: hybrid_uss
INFO - SETTING chunk_training_mode: disabled
INFO - SETTING multiprocess: None
INFO - SETTING num_processes: None
INFO - SETTING resume_after: None
INFO - SETTING trace_hh_id: None
INFO - ENV MKL_NUM_THREADS: None
INFO - ENV OMP_NUM_THREADS: None
INFO - ENV OPENBLAS_NUM_THREADS: None
INFO - NUMPY blas_opt_info libraries: ['openblas', 'openblas']
INFO - NUMPY lapack_opt_info libraries: ['openblas', 'openblas']
INFO - run single process simulation
INFO - Time to execute open_pipeline : 0.029 seconds (0.0 minutes)
INFO - prelo

In [22]:
modelname = "trip_mode_choice"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)
# data.coefficients
# data.spec
# data.chooser_data
model.load_data()
model.doctor(repair_ch_av='-')
model.loglike()
# DO NOT ESTIMATE just apply probs
# model.maximize_loglike(method='SLSQP', options={"maxiter": 1000})
probs = [m.probability() for m in model._k_models]

req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
problem: chosen-but-not-available (2 issues)
problem: low-variance-data-co (1 issues)
problem: chosen-but-not-available (2 issues)
problem: low-variance-data-co (1 issues)
problem: chosen-but-not-available (1 issues)
problem: low-variance-data-co (1 issues)
problem: chosen-but-no

-10094.898223413013

In [124]:
probs_0 = pd.DataFrame(probs[0])
m_0 = model._k_models[0]

probs_0.index = m_0.dataframes.data_ch.index

In [110]:
display(m_0.dataframes.data_ch.index.unique().shape)
display(np.intersect1d(data.chooser_data.index.values, m_0.dataframes.data_ch.index.values).shape)

(2124,)

(2124,)

In [111]:
#data.chooser_data[['override_choice', 'override_choice_code']].drop_duplicates().reset_index(drop=True).set_index\
#    ('override_choice_code').to_dict()['override_choice']
mode_map = pd.DataFrame.from_dict(data.settings["CONSTANTS"]["I_MODE_MAP"], orient='index', columns=["mode_code"])\
    .reset_index().rename(columns={"index": "mode_name"})

In [127]:
tour_id_to_check = 3021985

display(data.chooser_data.loc[data.chooser_data.index == tour_id_to_check])
display(m_0.dataframes.data_ch.loc[m_0.dataframes.data_ch.index == tour_id_to_check])
display(probs_0.loc[probs_0.index == tour_id_to_check])

,trip_id,model_choice,override_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,...,drive_heavyrail_available_outbound,drive_heavyrail_available_inbound,drive_commuter_available_outbound,drive_commuter_available_inbound,walk_ferry_available,drive_ferry_available,distance,distance_walk_od,distance_bike_od,override_choice_code
tour_id,,,,,,,,,,,,,,,,,,,,,
3021985,24175881,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,11.50,4.14944,...,False,False,False,False,False,False,5.96,5.99,5.99,1
3021985,24175885,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,11.81,4.14944,...,False,False,False,False,False,False,5.99,5.99,5.99,1


,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
_caseid_,,,,,,,,,,,,,,,,,,,,,
3021985,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3021985,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
_caseid_,,,,,,,,,,,,,,,,,,,,,
3021985,0.999871,0.0,0.0,0.0,0.0,0.0,0.000006,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.368530e-06,0.000021,0.000100
3021985,0.999886,0.0,0.0,0.0,0.0,0.0,0.000006,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.692194e-07,0.000011,0.000096


## now use my code to calculate probabilities